<img width="8%" alt="Azure Machine Learning.jpg" src="https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/.github/assets/logos/Azure%20Machine%20Learning.jpg" style="border-radius: 15%">

# Azure Machine Learning - Univariate Timeseries Inference
<a href="https://bit.ly/3JyWIk6">Give Feedback</a> | <a href="https://github.com/jupyter-naas/awesome-notebooks/issues/new?assignees=&labels=bug&template=bug_report.md&title=Azure+Machine+Learning+-+Univariate+Timeseries+Inference:+Error+short+description">Bug report</a>

**Tags:** #azure #machinelearning #univariate #timeseries #inference #ml

**Author:** [Tobias Zwingmann](https://www.linkedin.com/in/tobias-zwingmann/)

**Last update:** 2023-07-31 (Created: 2023-07-20)

**Description:** This notebook provides an example of how to use Azure Machine Learning to perform univariate timeseries inference. It is useful for organizations that need to analyze and predict future trends in their data. It requires a timeseries forecasting model hosted on Microsoft Azure and deployed as a web service. (See references below).

**References:**
- [Azure Machine Learning Documentation](https://docs.microsoft.com/en-us/azure/machine-learning/)
- [Tutorial: Forecast demand with no-code automated machine learning in the Azure Machine Learning studio](https://learn.microsoft.com/en-us/azure/machine-learning/tutorial-automated-ml-forecast)
- [Univariate Timeseries Inference](https://en.wikipedia.org/wiki/Time_series#Univariate_time_series_models)

## Input

### Import libraries

In [ ]:
import pandas as pd
import requests
import json
import matplotlib.pyplot as plt
import naas

### Setup Variables
- `timestamp_col`: The name of the column with the timestamps. Must be the same as used in training.
- `data_init`: Init data to be converted to DataFrame.
- `timestamps_to_predict`: A list of the timestamps to predict. The timestamps must lie **after** the timestamps observed during training
- `api_url`: the http URL to the Azure forecasting model
- `api_key`: the access key for the model

In [ ]:
timestamp_col = "timeStamp"
data_init = {
    timestamp_col: ["2017-08-10 01:00:00", "2017-08-10 02:00:00", "2017-08-10 03:00:00", "2017-08-10 04:00:00", "2017-08-10 05:00:00", "2017-08-10 06:00:00"],
    "values": [6053.458, 6714.258, 6497.025, 6360.583, 6333.775, 6534.683]
}
timestamps_to_predict = [
    "2017-08-10 06:00:00", 
    "2017-08-10 07:00:00",
    "2017-08-10 08:00:00",
    "2017-08-10 09:00:00",
    "2017-08-10 10:00:00",
    "2017-08-10 11:00:00"
]
# Azure hosted model URL and key - replace these with your own values!
api_url = naas.secret.get("AZURE_API_URL") or "Paste your API URL"
api_key = naas.secret.get("AZURE_API_KEY") or "Paste your API Key"

## Model

### Get historical data

In [ ]:
# Historic data - adjust if needed for plotting
data = pd.DataFrame(data_init)
data

### Perform Univariate Timeseries Inference

Using the forecasting model, perform univariate timeseries inference on the data.

In [ ]:
def inference_request(timestamp_col, timestamps):
    # Bind columns to dataframe
    request_df = pd.DataFrame({timestamp_col: timestamps})

    req = {
      "Inputs": {
          "data": list(request_df.to_dict('records'))
          },
         "GlobalParameters": {
             "quantiles": [0.025, 0.975]
             }
         }

    # POST request - send JSON to API
    headers = {
        'Authorization': ("Bearer " + api_key), 
        'Content-Type': 'application/json'
    }
    result = requests.post(api_url, data = str.encode(json.dumps(req)), headers=headers)
    return result

def format_timestamps(timestamps):
  # Format timestamp according to Azure requirements
  return(pd.to_datetime(pd.Series(timestamps_to_predict)).dt.strftime("%Y-%m-%dT%H:%M:%S.%fZ"))

# Get predictions
result = inference_request(timestamp_col, format_timestamps(timestamps_to_predict))
result = json.loads(result.content)

# Convert result to dataframe
results_df = pd.DataFrame(result['Results'])

# Move prediction intervals to separate columns
lower = []
upper = []
for element in results_df['prediction_interval']:
    lower.append(json.loads(element)[0])
    upper.append(json.loads(element)[1])

results_df['prediction_interval_lower'] = lower
results_df['prediction_interval_upper'] = upper
results_df['forecast'] = results_df['forecast']
results_df[timestamp_col] = timestamps_to_predict
results_df = results_df.drop("index", axis = 1)
results_df = results_df.drop("prediction_interval", axis = 1)

## Output

### Return predictions only

In [ ]:
results_df

### Return historic data and predictions

In [ ]:
data = pd.concat([data, results_df])
data

### Display chart

In [ ]:
# Plot the original data and the inference data
timestamps = data[timestamp_col]
observed_values = data["values"]
forecast_values = data["forecast"]
lower = data['prediction_interval_lower']
upper = data['prediction_interval_upper']

plt.figure(figsize=(10, 6))
plt.plot(timestamps, observed_values, label='Observed', linewidth=2)
plt.plot(timestamps, forecast_values, label='Forecast', linewidth=2)
plt.fill_between(timestamps, lower, upper, color='lightgray', alpha=0.5, label='Prediction Interval')
plt.xlabel('Timestamps')
plt.ylabel('Values')
plt.title('Time Series with Forecast and Prediction Interval')
plt.legend()
plt.xticks(rotation = 45)
plt.show()